# Text Detection with EasyOCR

This notebook will demonstrate how to use `EasyOCR` to detect text in an image. EasyOCR is a Python library that can perform Optical Character Recognition (OCR) with a few lines of code.

## Installation

First, we need to install the `EasyOCR` library. You can install it using pip.

In [ ]:
# Install EasyOCR
%%capture
!pip install easyocr

## Initialize EasyOCR Reader

The `EasyOCR` library supports multiple languages. Here, we will initialize the reader with English and Thai.

**Note:** This language input is only used for the text *recognition* process but not for the text *detection* process.

In [ ]:
from easyocr import Reader

# Initialize the EasyOCR reader
reader = Reader(["en", "th"])

## Load and Display the Image

We will load the our document image and display it using jupyter notebook built-in `display` function.

In [ ]:
from PIL import Image

# Load the image from a path
image_path = "path/to/image.jpg"
image = Image.open(image_path)

display(image)

## Perform Text Detection

We will use the `readtext` method from the `EasyOCR` reader to detect text in the image. This method returns a list of results, where each result contains the bounding box, the detected text, and the confidence score.

```py
[[[x1, y1], [x2, y2], [x3, y3], [x4, y4]], 'Detected Text', 0.99]
```

In [ ]:
# Perform text detection
results = reader.readtext(image_path)

# Print first result just to see the output
bbox, text, confidence_score = results[0]

print(f"Text: {text}")
print(f"Confidence score: {confidence_score}")
print(f"Bounding box: {bbox}")

## Draw Bounding Boxes and Display the Image with Predictions

We will draw the bounding boxes around the detected text and display the image with the text predictions.

In [ ]:
from PIL import ImageDraw, ImageFont

FONT = ImageFont.truetype("../assets/THSarabun.ttf", size=20)


def draw_boxes(image: Image.Image, results: list[tuple[int, int, int, int], str, float]) -> None:
    """Draw bounding boxes and its information on the image."""
    # Create a drawing object
    draw = ImageDraw.Draw(image)

    # Draw each result on the image.
    for result in results:
        # Unpack the result.
        bbox, text, confidence_score = result

        # bbox is a four-point coordinate of the bounding box. [[x1, y1], [x2, y2], [x3, y3], [x4, y4]]
        # we need to convert it to PIL coordinates to draw the rectangle.
        # which is only two points, top-left and bottom-right. [x1, y1, x2, y2]

        pil_bbox = [bbox[0][0], bbox[0][1], bbox[2][0], bbox[2][1]]

        # 1. Draw the bounding box
        draw.rectangle(pil_bbox, outline="blue", width=2)
        # 2. Draw the text and confidence score e.g. 'Hello (0.72)'
        draw_text = f"{text} ({confidence_score:.2f})"

        # Place text at the top-left of the bounding box and shift it up by 20 pixels
        x = pil_bbox[0]
        y = pil_bbox[1] - 20
        draw.text((x, y), draw_text, fill="red", font=FONT)


# Create a copy of the image to draw on
image_with_boxes = image.copy()
draw_boxes(image_with_boxes, results)

display(image_with_boxes)

## Advanced

We can use `easyocr.Reader` to perform only text detection task by using the `detect` method. This allows us to use custom text recognition model rather than easyocr's default model.

In [ ]:
from easyocr import Reader

# Initialize the EasyOCR reader
detector = Reader([])  # Don't need to specify any languages because we only need to detect the text regions

# Perform text detection
batch_regions, _ = detector.detect(image_path)

# Because we only inference single image, we only need the first result
regions = batch_regions[0]

In [ ]:
# Print first result just to see the output
bbox = regions[0]

print(f"Bounding box: {bbox}")

Now we can draw the bounding boxes.

In [ ]:
from PIL import ImageDraw

# Create a copy of the image to draw on
image_with_boxes = image.copy()

# Create a drawing object
draw = ImageDraw.Draw(image_with_boxes)

# Draw each region on the image
for region in regions:
    # Rearrange the region to match the PIL coordinate system
    region = [region[0], region[2], region[1], region[3]]
    draw.rectangle(region, outline="blue", width=2)

display(image_with_boxes)

# Extract Information from the recognized text

## Create extract question

In [20]:
extract_question = """
You are provided with a recognized text from the OCR system of a Thai vehicle registration book (สมุดทะเบียนรถ), which each recognized text are seperated by tab (\t) character.
Your task is to extract the following information from the image.
The extracted value is typically located on the right side of the key in the document.
Some of the text might be corrupted, missing diacritics or misread, autocorrection is appreciated.
Extract these details:

1. วันจดทะเบียน (date_of_registration)
2. เลขทะเบียน (registration_no)
3. จังหวัด (car_province)
4. ประเภท (vehicle_use)
5. รย. (type)
6. ลักษณะ (body_style)
7. ยี่ห้อรถ (manufacturer)
8. แบบ (model)
9. รุ่นปี คศ (year)
10. สี (color)
11. เลขตัวรถ (chassis_number)
12. อยู่ที่ (chassis_location)
13. ยี่ห้อเครื่องยนต์ (engine_manufacturer)
14. เลขเครื่องยนต์ (engine_number)
15. อยู่ที่ (engine_location)
16. เชื้อเพลิง (fuel_type)
17. เลขถังแก๊ส (fuel_tank_number)
18. จำนวน (cylinders)
19. ซีซี (cubic_capacity)
20. แรงม้า (horse_power)
21. จำนวนเพลาและล้อ (axles_wheels_no)
22. น้ำหนักรถ (unladen_weight)
23. น้ำหนักบรรทุก/น้ำหนักเพลา (load_capacity)
24. น้ำหนักรวม (gross_weight)
25. ที่นั่ง (seats)

Instructions:

Carefully examine the image and locate each piece of information.
If a particular field is not visible or not present in the image, use the value "N/A" for that field.
Ensure all text extracted from the image is in its original language (Thai or English) as it appears in the document.
Return the extracted information in a JSON format, using the English key names provided in parentheses.
Only return the JSON output, without any additional explanation or text.

Example of expected output in dictionary format:
{
  "date_of_registration": "1 ม.ค. 2566",
  "registration_no": "กข 1234",
  "car_province": "กรุงเทพมหานคร",
  ...
  "seats": "4"
}
"""

## Perform information extraction with Llama3.2

In [ ]:
# Stack recognized text into a single string for prompting
recognized_text = "\t".join([result[1] for result in results])
recognized_text

In [10]:
from langchain_ollama import OllamaLLM
from langchain import PromptTemplate

llm = OllamaLLM(model="llama3.2", stop=["<|eot_id|>"]) # Added stop token

def get_model_response(user_prompt: str, system_prompt: str) -> str:
    # NOTE: No f string and no whitespace in curly braces
    template = """
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        {system_prompt}
        <|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        {user_prompt}
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        """

    # Added prompt template
    prompt = PromptTemplate(
        input_variables=["system_prompt", "user_prompt"],
        template=template
    )

    # Modified invoking the model
    response = llm.invoke(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))

    return response

In [21]:
# Example
user_prompt = recognized_text
system_prompt = extract_question
answer = get_model_response(user_prompt, system_prompt)

In [ ]:
print(answer)